In [1]:
from boto3.s3.transfer import S3Transfer
import boto3
import os
import zipfile
from zipfile import ZipFile as zf
import pandas as pd

In [2]:
# S3 access, bucket and file configurations
aws_access_key_id = 'XXXX'
aws_secret_access_key = 'XXXX'
s3 = boto3.resource('s3')
s3_bucket_name = 'erho2'
file_downloaded = "Olist.zip"

In [3]:
# Download dataset from S3
s3 = boto3.client('s3')
s3.download_file(s3_bucket_name, file_downloaded, file_downloaded)

In [2]:
# Unzip dataset which I downloaded from S3
with zf(file_downloaded, 'r') as zipObj:
    zipObj.extractall()

NameError: name 'file_downloaded' is not defined

In [3]:
# Dataset reading
customer = pd.read_csv("olist_customers_dataset.csv")
geolocation = pd.read_csv("olist_geolocation_dataset.csv")
order_items = pd.read_csv("olist_order_items_dataset.csv")
order_payments = pd.read_csv("olist_order_payments_dataset.csv")
order_reviews = pd.read_csv("olist_order_reviews_dataset.csv")
orders = pd.read_csv("olist_orders_dataset.csv")
products = pd.read_csv("olist_products_dataset.csv")
sellers = pd.read_csv("olist_sellers_dataset.csv")
translation = pd.read_csv("product_category_name_translation.csv") # We don't need it but I would like to read all of CSV files

In [4]:
# Dataset information
datasets = [customer, geolocation, order_items, order_payments, order_reviews, orders, products, sellers]
titles = ["customers","geolocation", "items", "payments", "reviews", "orders", "products", "sellers"]
info_df = pd.DataFrame({},)
info_df["Dataset"] = titles
info_df['Columns'] = [','.join([col for col, null in df.isnull().sum().items()]) for df in datasets]
info_df["Columns No"] = [df.shape[1] for df in datasets]
info_df["Null Amount"] = [df.isnull().sum().sum() for df in datasets]
info_df["Null Columns Amount"] = [len([col for col, null in df.isnull().sum().items() if null > 0]) for df in datasets]
info_df['Null Cols'] = [', '.join([col for col, null in df.isnull().sum().items() if null > 0]) for df in datasets]

info_df.style.background_gradient(cmap='coolwarm')



,Dataset,Columns,Columns No,Null Amount,Null Columns Amount,Null Cols
0,customers,"customer_id,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state",5,0,0,
1,geolocation,"geolocation_zip_code_prefix,geolocation_lat,geolocation_lng,geolocation_city,geolocation_state",5,0,0,
2,items,"order_id,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value",7,0,0,
3,payments,"order_id,payment_sequential,payment_type,payment_installments,payment_value",5,0,0,
4,reviews,"review_id,order_id,review_score,review_comment_title,review_comment_message,review_creation_date,review_answer_timestamp",7,145903,2,"review_comment_title, review_comment_message"
5,orders,"order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date",8,4908,3,"order_approved_at, order_delivered_carrier_date, order_delivered_customer_date"
6,products,"product_id,product_category_name,product_name_lenght,product_description_lenght,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm",9,2448,8,"product_category_name, product_name_lenght, product_description_lenght, product_photos_qty, product_weight_g, product_length_cm, product_height_cm, product_width_cm"
7,sellers,"seller_id,seller_zip_code_prefix,seller_city,seller_state",4,0,0,


In [5]:
# Merge all tables and check missing values and types of datas
df = pd.merge(orders,order_payments, on="order_id")
df = df.merge(customer, on="customer_id")
df = df.merge(order_items, on="order_id")
df = df.merge(products, on="product_id")
df = df.merge(translation, on="product_category_name")
df.dropna(inplace=True) # Null variables have gone
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 113367 entries, 0 to 115877
Data columns (total 31 columns):
 #   Column                         Non-Null Count   Dtype  
---  ------                         --------------   -----  
 0   order_id                       113367 non-null  object 
 1   customer_id                    113367 non-null  object 
 2   order_status                   113367 non-null  object 
 3   order_purchase_timestamp       113367 non-null  object 
 4   order_approved_at              113367 non-null  object 
 5   order_delivered_carrier_date   113367 non-null  object 
 6   order_delivered_customer_date  113367 non-null  object 
 7   order_estimated_delivery_date  113367 non-null  object 
 8   payment_sequential             113367 non-null  int64  
 9   payment_type                   113367 non-null  object 
 10  payment_installments           113367 non-null  int64  
 11  payment_value                  113367 non-null  float64
 12  customer_unique_id            

In [6]:
# Correction data types of dates
df['order_purchase_timestamp'] = pd.to_datetime(df.order_purchase_timestamp)
df['order_approved_at'] = pd.to_datetime(df.order_approved_at)
df['order_delivered_carrier_date'] = pd.to_datetime(df.order_delivered_carrier_date)
df['order_delivered_customer_date'] = pd.to_datetime(df.order_delivered_customer_date)
df['order_estimated_delivery_date'] = pd.to_datetime(df.order_estimated_delivery_date)
df['shipping_limit_date'] = pd.to_datetime(df.shipping_limit_date)

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 113367 entries, 0 to 115877
Data columns (total 31 columns):
 #   Column                         Non-Null Count   Dtype         
---  ------                         --------------   -----         
 0   order_id                       113367 non-null  object        
 1   customer_id                    113367 non-null  object        
 2   order_status                   113367 non-null  object        
 3   order_purchase_timestamp       113367 non-null  datetime64[ns]
 4   order_approved_at              113367 non-null  datetime64[ns]
 5   order_delivered_carrier_date   113367 non-null  datetime64[ns]
 6   order_delivered_customer_date  113367 non-null  datetime64[ns]
 7   order_estimated_delivery_date  113367 non-null  datetime64[ns]
 8   payment_sequential             113367 non-null  int64         
 9   payment_type                   113367 non-null  object        
 10  payment_installments           113367 non-null  int64         
 11  

In [8]:
# Merge "id" column from orders table by order_id column, 
# Merge "product_id", "product_category_name", "product_id" columns from products table by product_id column
# Merged these tables for calculate late orders
merged_order_products = order_items.merge(orders, on="order_id").merge(products[["product_id", "product_category_name"]], on="product_id")
late_orders = merged_order_products.loc[merged_order_products.shipping_limit_date < merged_order_products.order_delivered_carrier_date].head()

In [9]:
# Late orders list
late_orders

,order_id,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,product_category_name
0,00010242fe8c5a6d1ba2dd792cb16214,1,4244733e06e7ecb4970a6e2683c13e61,48436dade18ac8b2bce089ec2a041202,2017-09-19 09:45:35,58.9,13.29,3ce436f183e68e07877b285a838db11a,delivered,2017-09-13 08:59:02,2017-09-13 09:45:35,2017-09-19 18:34:16,2017-09-20 23:43:48,2017-09-29 00:00:00,cool_stuff
1,130898c0987d1801452a8ed92a670612,1,4244733e06e7ecb4970a6e2683c13e61,48436dade18ac8b2bce089ec2a041202,2017-07-05 02:44:11,55.9,17.96,e6eecc5a77de221464d1c4eaff0a9b64,delivered,2017-06-28 11:52:20,2017-06-29 02:44:11,2017-07-05 12:00:33,2017-07-13 20:39:29,2017-07-26 00:00:00,cool_stuff
2,532ed5e14e24ae1f0d735b91524b98b9,1,4244733e06e7ecb4970a6e2683c13e61,48436dade18ac8b2bce089ec2a041202,2018-05-23 10:56:25,64.9,18.33,4ef55bf80f711b372afebcb7c715344a,delivered,2018-05-18 10:25:53,2018-05-18 12:31:43,2018-05-23 14:05:00,2018-06-04 18:34:26,2018-06-07 00:00:00,cool_stuff
9,00018f77f2f0320c557190d7a144bdd3,1,e5f2d52b802189ee658865ca93d83a8f,dd7ddc04e1b6c2c614352b383efe2d36,2017-05-03 11:05:13,239.9,19.93,f6dd3ec061db4e3987629fe6b26e5cce,delivered,2017-04-26 10:53:06,2017-04-26 11:05:13,2017-05-04 14:35:00,2017-05-12 16:04:24,2017-05-15 00:00:00,pet_shop
15,00042b26cf59d7ce69dfabb4e55b4fd9,1,ac6c3623068f30de03045865e4e10089,df560393f3a51e74553ab94004ba5c87,2017-02-13 13:57:51,199.9,18.14,58dbd0b2d70206bf40e62cd34e84d795,delivered,2017-02-04 13:57:51,2017-02-04 14:10:13,2017-02-16 09:46:09,2017-03-01 16:42:31,2017-03-17 00:00:00,ferramentas_jardim


In [10]:
# Also we can use this commands provide types of data for each column BUT I don't know why it gives me a new baby column named "c0" xD You can check with codes below
# late_orders.to_csv("late_orders.csv")
# spark_late = sc.read.csv("late_orders.csv", inferSchema=True, header=True)
# spark_late.printSchema()

In [11]:
import findspark
findspark.init()
import pyspark
from pyspark import SparkContext
from pyspark import SparkConf
from pyspark.sql import SQLContext
from pyspark.sql import *
from pyspark.sql.functions import *
spark = SparkSession.builder \
.master("local[1]") \
.appName("") \
.getOrCreate()
sql = SQLContext(spark)

C:\Bigdata\spark\python\pyspark\sql\context.py:77: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


In [12]:
spark.conf.set("spark.sql.execution.arrow.enabled", "true")
spark_late_items = sql.createDataFrame(merged_order_products)
spark_late_items.createOrReplaceTempView('spark_late_items')
spark_late_items.show(5)

+--------------------+-------------+--------------------+--------------------+-------------------+-----+-------------+--------------------+------------+------------------------+-------------------+----------------------------+-----------------------------+-----------------------------+---------------------+
|            order_id|order_item_id|          product_id|           seller_id|shipping_limit_date|price|freight_value|         customer_id|order_status|order_purchase_timestamp|  order_approved_at|order_delivered_carrier_date|order_delivered_customer_date|order_estimated_delivery_date|product_category_name|
+--------------------+-------------+--------------------+--------------------+-------------------+-----+-------------+--------------------+------------+------------------------+-------------------+----------------------------+-----------------------------+-----------------------------+---------------------+
|00010242fe8c5a6d1...|            1|4244733e06e7ecb49...|48436dade18ac8b2

In [13]:
spark.sql('SELECT * FROM spark_late_items').show(5)

+--------------------+-------------+--------------------+--------------------+-------------------+-----+-------------+--------------------+------------+------------------------+-------------------+----------------------------+-----------------------------+-----------------------------+---------------------+
|            order_id|order_item_id|          product_id|           seller_id|shipping_limit_date|price|freight_value|         customer_id|order_status|order_purchase_timestamp|  order_approved_at|order_delivered_carrier_date|order_delivered_customer_date|order_estimated_delivery_date|product_category_name|
+--------------------+-------------+--------------------+--------------------+-------------------+-----+-------------+--------------------+------------+------------------------+-------------------+----------------------------+-----------------------------+-----------------------------+---------------------+
|00010242fe8c5a6d1...|            1|4244733e06e7ecb49...|48436dade18ac8b2

In [14]:
spark_late_items.printSchema()

root
 |-- order_id: string (nullable = true)
 |-- order_item_id: long (nullable = true)
 |-- product_id: string (nullable = true)
 |-- seller_id: string (nullable = true)
 |-- shipping_limit_date: string (nullable = true)
 |-- price: double (nullable = true)
 |-- freight_value: double (nullable = true)
 |-- customer_id: string (nullable = true)
 |-- order_status: string (nullable = true)
 |-- order_purchase_timestamp: string (nullable = true)
 |-- order_approved_at: string (nullable = true)
 |-- order_delivered_carrier_date: string (nullable = true)
 |-- order_delivered_customer_date: string (nullable = true)
 |-- order_estimated_delivery_date: string (nullable = true)
 |-- product_category_name: string (nullable = true)



In [15]:
spark_late_items.coalesce(1).write.option("header", "true").csv("spark_late_items")

Py4JJavaError: An error occurred while calling o47.csv.
: org.apache.spark.SparkException: Job aborted.
	at org.apache.spark.sql.errors.QueryExecutionErrors$.jobAbortedError(QueryExecutionErrors.scala:496)
	at org.apache.spark.sql.execution.datasources.FileFormatWriter$.write(FileFormatWriter.scala:251)
	at org.apache.spark.sql.execution.datasources.InsertIntoHadoopFsRelationCommand.run(InsertIntoHadoopFsRelationCommand.scala:186)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.sideEffectResult$lzycompute(commands.scala:113)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.sideEffectResult(commands.scala:111)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.executeCollect(commands.scala:125)
	at org.apache.spark.sql.execution.QueryExecution$$anonfun$eagerlyExecuteCommands$1.$anonfun$applyOrElse$1(QueryExecution.scala:110)
	at org.apache.spark.sql.execution.SQLExecution$.$anonfun$withNewExecutionId$5(SQLExecution.scala:103)
	a